# Simple CNN model for classification 

In [1]:
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import applications

Using TensorFlow backend.


In [2]:
# dimensions of our images.
img_width, img_height = 299, 299

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
nb_train_samples = 1902
nb_validation_samples = 98
epochs = 50
batch_size = 16

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('top_model.h5')

Found 1902 images belonging to 2 classes.
Found 98 images belonging to 2 classes.
Epoch 1/50
118/118 [==============================] - 91s - loss: 0.7632 - acc: 0.5185 - val_loss: 0.6825 - val_acc: 0.6979
Epoch 2/50
118/118 [==============================] - 80s - loss: 0.6963 - acc: 0.5329 - val_loss: 0.6737 - val_acc: 0.6829
Epoch 3/50
118/118 [==============================] - 77s - loss: 0.6810 - acc: 0.5711 - val_loss: 0.6531 - val_acc: 0.5854
Epoch 4/50
118/118 [==============================] - 77s - loss: 0.6831 - acc: 0.5949 - val_loss: 0.6340 - val_acc: 0.6341
Epoch 5/50
118/118 [==============================] - 85s - loss: 0.6689 - acc: 0.6167 - val_loss: 0.7459 - val_acc: 0.5854
Epoch 6/50
118/118 [==============================] - 81s - loss: 0.6634 - acc: 0.6234 - val_loss: 0.6611 - val_acc: 0.6585
Epoch 7/50
118/118 [==============================] - 78s - loss: 0.6630 - acc: 0.6369 - val_loss: 0.7136 - val_acc: 0.6341
Epoch 8/50
118/118 [==============================

# Extract features from a pretrained model

In [2]:
import numpy as np
import h5py

In [3]:
# dimensions of our images.
img_width, img_height = 299, 299

train_data_dir = 'data/train'
validation_data_dir = 'data/test'
top_model_weights_path = 'top_model.h5'

nb_train_samples = 1900
nb_validation_samples = 100
epochs = 50
batch_size = 10

if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [4]:
def save_bottelneck_features():
    
    train_datagen = ImageDataGenerator(rescale=1/255)
    train_generator = train_datagen.flow_from_directory(train_data_dir,target_size=[299,299],
                                                       batch_size=batch_size, class_mode=None,
                                                       shuffle=False)
    
    model = applications.VGG16(weights='imagenet',include_top=False)
    bottelneck_features_train = model.predict_generator(train_generator, nb_train_samples // batch_size)
    np.save(open('bottelneck_train_features.npy','wb'), bottelneck_features_train)
    
    validation_datagen = ImageDataGenerator(rescale=1/255)
    validation_generator = validation_datagen.flow_from_directory(validation_data_dir, target_size=[299,299],
                                                                 batch_size= batch_size, class_mode=None,
                                                                 shuffle=False)
    
    bottelneck_features_validation = model.predict_generator(validation_generator,
                                                             nb_validation_samples // batch_size)
    np.save(open('bottelneck_features_validation.npy','wb'),bottelneck_features_validation)
    
def train_model():
    
    train_data = np.load(open('bottelneck_train_features.npy','rb'))
    train_labels = np.array([0]*int(nb_train_samples /2) + [1]*int(nb_train_samples/2))
    
    validation_data = np.load(open('bottelneck_features_validation.npy','rb'))
    validation_labels = np.array([0]*int(nb_validation_samples/2) + [1]*int(nb_validation_samples/2))
    
    
    model = Sequential()
    model.add(Flatten(input_shape=train_data.shape[1:]))
    model.add(Dense(256,activation='relu'))
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.6))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='rmsprop',
                  loss='binary_crossentropy', metrics=['accuracy'])
    
    model.fit(train_data, train_labels,
              epochs=epochs,
              batch_size=batch_size,
              validation_data=(validation_data, validation_labels))
    
    model.save(top_model_weights_path)
    


In [14]:
save_bottelneck_features()

In [6]:
train_data = np.load(open('bottelneck_train_features.npy','rb'))
train_labels = np.array([0]*int(nb_train_samples /2) + [1]*int(nb_train_samples/2))

In [12]:
train_data.shape, train_labels.shape

((1900, 9, 9, 512), (1900,))

In [38]:
validation_data = np.load(open('bottelneck_features_validation.npy','rb'))
validation_labels = np.array([0]*int(nb_validation_samples/2) + [1]*int(nb_validation_samples/2))

In [39]:
validation_data.shape, validation_labels.shape

((100, 9, 9, 512), (100,))

In [5]:
train_model()

Train on 1900 samples, validate on 100 samples
Epoch 1/50
1900/1900 [==============================] - 22s - loss: 2.1168 - acc: 0.5184 - val_loss: 0.6931 - val_acc: 0.5000
Epoch 2/50
1900/1900 [==============================] - 21s - loss: 0.7674 - acc: 0.5205 - val_loss: 0.6975 - val_acc: 0.5000
Epoch 3/50
1900/1900 [==============================] - 21s - loss: 0.7337 - acc: 0.5305 - val_loss: 0.8208 - val_acc: 0.5200
Epoch 4/50
1900/1900 [==============================] - 21s - loss: 0.6724 - acc: 0.6342 - val_loss: 0.6467 - val_acc: 0.6400
Epoch 5/50
1900/1900 [==============================] - 21s - loss: 0.5864 - acc: 0.7211 - val_loss: 0.5641 - val_acc: 0.7100
Epoch 6/50
1900/1900 [==============================] - 21s - loss: 0.4634 - acc: 0.7779 - val_loss: 0.4849 - val_acc: 0.8100
Epoch 7/50
1900/1900 [==============================] - 21s - loss: 0.3890 - acc: 0.8300 - val_loss: 0.4813 - val_acc: 0.8400
Epoch 8/50
1900/1900 [==============================] - 21s - loss: 0.3